In [ ]:

import os

from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

load_dotenv()

In [21]:
# Transform loader to pages 
loader = PyPDFLoader("pdfs/10.1002@bscb.19810900913.pdf")
pages = loader.load_and_split()

In [23]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])
faiss = FAISS.from_documents(pages, embeddings)

<bound method FAISS.similarity_search of <langchain_community.vectorstores.faiss.FAISS object at 0x127bbd700>>


In [48]:
question = "What is the molecule of the paper?"

docs_db = faiss.similarity_search(question, k=5)
print(len(docs_db))

5


In [49]:
print(docs_db[1])

page_content="seEaflrion-pf-The_-s_Te_Nl-lc_e_T~Te_s_ \n103 mg of the steryl acetates were chromatographed several times on columns \nof silica gel impregnated with 15% AgN03 using a gradient elution of benzene \n(from 3 to 50%) in hexane). Eluted fractions were monitored by GC analysis. \nCombined fractions were finally purified by preparative tlc (eluent : hexane- \nbenzene 1:l) : the silica band corresponding to Rf 0.4 to 0.6 was extracted by \na 1:l mixture of CHC13 and AcOEt. This yielded 3 mg of 2-1 (IIIb), 77 mg Of 2-2 \n(In) and 8 mg Of 2-3 (Vb). \nCompound IIIb : : 95% pure, RRT = 1.00, co-chromatographs with authentic \n6 \n: 1730, 1640, 1250 and 895 IIIb. : M+-60 = 368, 353 (368-CH;) ,... \nCompound IV b : : 98% pure, RRT = 1.31. m.p. = 133-1340 (lit. 134-1350) . \n[alD = -42.80 (lit. -46.7?)6. \ncm-'. !g = M+-60 = 380, 365 (380-CH3), 296 (Mac Lafferty from \n380), 281, 255, 253 (380-side chain + 2H), 228 and 213. 'H NMR \n(CDC13) : 0.65 (3H s C-18) - 0.95 (3H d J=6.5 HZ C-2

In [36]:
dir(docs_db)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [52]:
from openai import OpenAI

client = OpenAI()

client.api_key = os.environ["OPENAI_API_KEY"]
stream = client.chat.completions.create(
    messages=[{"role": "system",
               "content": "You are a chemist expert in natural products. You give the answer in JSON format. Answer user's questions utilizing your background knowledge or the information given below"},
              {"role": "user", "content": docs_db[1]}],
    stream=True,
    response_format={"type": "json_object"},
    model = "gpt-4-1106-preview",
)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

BadRequestError: Error code: 400 - {'error': {'message': "'$.messages[1].content' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [33]:

print(stream.response.read().json(
))

StreamConsumed: Attempted to read or stream some content, but the content has already been streamed. For requests, this could be due to passing a generator as request content, and then receiving a redirect response or a secondary request as part of an authentication flow.For responses, this could be due to attempting to stream the response content more than once.

In [54]:
# nbstripout is a tool to remove the output from Jupyter notebooks
!nbstripout --install